In [1]:
import numpy as np
import pandas as pd
import scipy
import scipy.special
import openpyxl
import os
import scipy.stats as stats
import pyodbc
from datetime import datetime, timedelta
from collections import OrderedDict

##UPDATE LOG
## In January 2021, Vizient updated their Report Express to allow both a numerator and denominator file.  Previously,
## Vizient only provided a final rate and numerator file.  This meant we could not precisely control the time periods
## since Vizient only allowed file downloads by quarter.  Now, with the file updates, we can download both numerator
## and denominator files as well as filter records by discharge month.  Now we can control the time period.

## This python script will be used to process all the files.  

## NOTE:  Vizient's user interface currently only allows file downloads of max 5 hospitals at a time.  Northwestern Memorial 
## has 8 Vizient hospitals.  Therefore, for each lab type, we need to download 2 files and then union the data
## back together.

## NOTE 2:  Within each zipped lab data folder, there will be 4 files:
##          1.  _Completeness.csv  (for example, Hypoglycemia_Completeness.csv).  This file indicates whether or not the data 
##              download was successful.  For each hospital, Vizient indicates whether or not there was a problem with the data.
##          2.  _Denominator_PHI.csv.  This file holds all the denominator patient records.  This is our denominator file.
##          3.  _Numerator_PHI.csv.  This file holds all the numerator patient records.  This is our numerator detail file.
##          4.  _QA.csv.  This file holds the final summary result with numerator, denominator and rate.  However, the 
##              calculation is quarterly.  We originally used this file to get the results but now we can 
##              calculate the numbers ourselves and filter by the exact time period we want so we will no longer use this file.
##              However, this file could be used for validation.


## NOTE 3:  In 2020, Critical Access hospital do not have lab measures.  We did not need to do this for Valley West because
##          Valley West is currently the only Critical Access cohort hospital in NM's system.


### Set Filepath for Lab Data folder

In [2]:
filepath2 = r'P:\Datastore02\Analytics\230 Inpatient Quality Composite\data\Lab Data\fy23\010_june_fytd'

### Get list of all NM hospital medicare IDs to filter the dataset with

In [3]:
calculator_id = 15

# query the db to get period id
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=edw00pd05wva.corp.nm.org\EDWIDS1;'
                      'Database=clarity;'
                      'Trusted_Connection=yes;')
nm_medicare_ids = """
SELECT
h.hospital_medicare_id
,c.cohort_name
FROM
NM_Analytics_Prototype.vizient_qa.hospitals as h
join NM_Analytics_Prototype.vizient_qa.calc_hospital_cohort as chc
on chc.hospital_id = h.hospital_id
join NM_Analytics_Prototype.vizient_qa.cohort as c
on c.cohort_id = chc.hospital_cohort_id
where
left(h.hospital_name, 12) = 'NORTHWESTERN'
AND
chc.calc_id = %s
AND
c.cohort_name in ('Comprehensive Academic Medical Center','Large Specialized Complex Care Medical Center','Complex Care Medical Center','Community')
""" % calculator_id

query_results = pd.DataFrame(pd.read_sql(nm_medicare_ids, conn))

conn.close()
nm_medicare_id_list = list(query_results.set_index('hospital_medicare_id').to_dict()['cohort_name'].keys())
#convert medicare id strings to integers because the csv file lists them in integer format.
nm_medicare_id_ints = [int(i) for i in nm_medicare_id_list if i != 140130]

### Query the database and get the current IQC Period ID based on last month's end date.  
### Update period_end_dts

In [4]:
period_end_dts = '06-30-2023 23:59:59'
period_type = 'NM_FSCL_YTD'

# query the db to get period id
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=edw00pd05wva.corp.nm.org\EDWIDS1;'
                      'Database=clarity;'
                      'Trusted_Connection=yes;')
period_ids = """
select
period_id
FROM
NM_Analytics_Prototype.vizient_qa.periods as p
join NM_Analytics_Prototype.vizient_qa.period_types as pt
on pt.period_type_id = p.period_type
where
p.period_end_date = '%s'
AND
pt.period_type_nm = '%s'
""" % (period_end_dts,period_type)

period_query_results = pd.DataFrame(pd.read_sql(period_ids, conn))['period_id'][0]

conn.close()

### Validate the period id is correct (as expected)

In [5]:
period_query_results

7514

### Create a dictionary of hospital cohorts and hospital medicare IDS

In [6]:
hospital_cohort_dict = query_results.set_index('hospital_medicare_id').to_dict()['cohort_name']

In [7]:
#hospital_cohort_dict.pop('140062')

In [8]:
#nm_medicare_id_ints.remove(140062)

In [9]:
nm_medicare_id_ints

[140281, 140242, 140130, 140211, 140286, 149916, 140116, 140062]

### Set Parameters for Baseline calculator, Fiscal Year and Period End Dts
### Update period_end_dts

In [10]:
calc_nm_input = '2022 Q&A calculator Period 3'
period_type_input = 'NM_FSCL_YTD'
period_end_dts_input = '06/30/2023 23:59:59'

### Get list of discharge months to filter the dataset

In [11]:
start_dts = input('Input the start discharge month (using format yyyy-mm-dd)')

Input the start discharge month (using format yyyy-mm-dd)2022-08-01


In [12]:
end_dts = input('Input the end discharge month (using format yyyy-mm-dd)')

Input the end discharge month (using format yyyy-mm-dd)2023-05-01


In [13]:
keep_discharge_months = pd.date_range(start_dts,end_dts, freq='MS').strftime("%Y%m").tolist()

In [14]:
keep_discharge_months

['202208',
 '202209',
 '202210',
 '202211',
 '202212',
 '202301',
 '202302',
 '202303',
 '202304',
 '202305']

### Create a dataframe of all Lab measures, IDs and domain name

In [15]:
# query the db to get equity measure ids for later join
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=edw00pd05wva.corp.nm.org\EDWIDS1;'
                      'Database=clarity;'
                      'Trusted_Connection=yes;')
lab_meas = """
select
distinct
m.measure_id
,m.measure_name
,d.domain_nm
from
NM_Analytics_Prototype.vizient_qa.measure as m
join NM_Analytics_Prototype.vizient_qa.domain as d
on d.domain_id = m.domain_id
join NM_Analytics_Prototype.vizient_qa.calc_measure_values as cmv
on cmv.measure_id = m.measure_id
where
d.domain_nm in ('Safety','Effectiveness')
and
m.measure_id
IN
(
30   --Sepsis lactate	
,31   --Transfusion	
,172 --Warfarin elevated INR	
,158 --Hypoglycemia in insulin use	
)
AND
cmv.calc_id = %s
""" % calculator_id

lab_measure_df = pd.DataFrame(pd.read_sql(lab_meas, conn))

conn.close()

In [16]:
lab_measure_df

,measure_id,measure_name,domain_nm
0,30,Sepsis lactate,Effectiveness
1,31,Transfusion,Effectiveness
2,158,Hypoglycemia in insulin use,Safety
3,172,Warfarin elevated INR,Safety


### Query the database to get current hospital cohort assignments

In [17]:
# query the db to get equity measure ids for later join
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=edw00pd05wva.corp.nm.org\EDWIDS1;'
                      'Database=clarity;'
                      'Trusted_Connection=yes;')
hosp_query = """
select
h.hospital_id
,h.hospital_medicare_id
FROM
NM_Analytics_Prototype.vizient_qa.hospitals as h
join NM_Analytics_Prototype.vizient_qa.calc_hospital_cohort as chc
on chc.hospital_id = h.hospital_id
WHERE
chc.calc_id = %s
""" % calculator_id

hospital_df = pd.DataFrame(pd.read_sql(hosp_query, conn))

conn.close()

In [18]:
hospital_df

,hospital_id,hospital_medicare_id
0,0,0
1,1,010033
2,2,030064
3,3,040016
4,4,050025
...,...,...
768,803,170109
769,804,251313
770,805,251315
771,806,251331


In [19]:
hospital_cohort_dict

{'140281': 'Comprehensive Academic Medical Center',
 '140242': 'Large Specialized Complex Care Medical Center',
 '140130': 'Complex Care Medical Center',
 '140211': 'Complex Care Medical Center',
 '140286': 'Complex Care Medical Center',
 '149916': 'Complex Care Medical Center',
 '140116': 'Complex Care Medical Center',
 '140062': 'Complex Care Medical Center'}

### UL001 

In [20]:
lab_measure_df

,measure_id,measure_name,domain_nm
0,30,Sepsis lactate,Effectiveness
1,31,Transfusion,Effectiveness
2,158,Hypoglycemia in insulin use,Safety
3,172,Warfarin elevated INR,Safety


In [21]:
result_list = []   

lactate_num_list = []
lactate_denom_list = []
hypo_num_list = []
hypo_denom_list = []
warfarin_num_list = []
warfarin_denom_list = []
trans_num_list = []
trans_denom_list = []

    
for i,item in enumerate(os.listdir(filepath2)):
    if item.endswith('.zip') == False and (item.endswith('.xlsx') == False):
        #print(item)
        #open each phi file and sort them into correct buckets
        denom_phi_list = [i for i in os.listdir(os.path.join(filepath2,item)) if 'Denominator_PHI' in i]
        numer_phi_list = [i for i in os.listdir(os.path.join(filepath2,item)) if 'Numerator_PHI' in i]
        file_type_denom = denom_phi_list[0].split('_')[0].replace(' ','').upper()
        file_type_num = numer_phi_list[0].split('_')[0].replace(' ','').upper()
        
        if file_type_denom == 'LACTATE':
            lactate_denom_list.append(os.path.join(filepath2,item,denom_phi_list[0]))
        if file_type_num == 'LACTATE':
            lactate_num_list.append(os.path.join(filepath2,item,numer_phi_list[0]))
        if file_type_denom == 'HYPOGLYCEMIA':
            hypo_denom_list.append(os.path.join(filepath2,item,denom_phi_list[0]))
        if file_type_num == 'HYPOGLYCEMIA':
            hypo_num_list.append(os.path.join(filepath2,item,numer_phi_list[0]))
        if file_type_denom == 'TRANSFUSION':
            trans_denom_list.append(os.path.join(filepath2,item,denom_phi_list[0]))
        if file_type_num == 'TRANSFUSION':
            trans_num_list.append(os.path.join(filepath2,item,numer_phi_list[0]))
        if file_type_denom == 'WARFARIN':
            warfarin_denom_list.append(os.path.join(filepath2,item,denom_phi_list[0]))
        if file_type_num == 'WARFARIN':
            warfarin_num_list.append(os.path.join(filepath2,item,numer_phi_list[0]))
            
#create some containers for unions later on.
list_of_lists = [lactate_num_list\
,lactate_denom_list \
,hypo_num_list \
,hypo_denom_list\
,warfarin_num_list\
,warfarin_denom_list\
,trans_num_list\
,trans_denom_list\
]

lactate_num_list2 = []
lactate_denom_list2 = []
hypo_num_list2 = []
hypo_denom_list2 = []
warfarin_num_list2 = []
warfarin_denom_list2 = []
trans_num_list2 = []
trans_denom_list2 = []


for i, item in enumerate(list_of_lists):
    for j, item2 in enumerate(item):
        print(item2)
        #read in the csv file.  The files should not have a header so default read_csv should work.
        try:
            df = pd.read_csv(item2)
            if 'Medicareid' in df.columns:
                df.rename(columns = {'Medicareid':'MedicareId'}, inplace = True) 

            #filter by discharge month
            df = df[df['dischargemonth'].isin(keep_discharge_months)]
            
            #Pivot and COUNT DISTINCT RecordID column.
            df = df.groupby('MedicareId').RecordId.nunique().reset_index()
            #print(df.head())
            if 'lactate' in item2.lower() and 'numerator' in item2.lower():
                df['measure'] = 'Sepsis lactate'
                df.rename(columns = {'RecordId':'numerator'}, inplace = True) 
                df['numerator_event_type_id'] = 22
                #print(df.head())
                lactate_num_list2.append(df)

            if 'lactate' in item2.lower() and 'denominator' in item2.lower():
                df['measure'] = 'Sepsis lactate'
                df.rename(columns = {'RecordId':'denominator'}, inplace = True) 
                df['denominator_event_type_id'] = 23
                #print(df)
                lactate_denom_list2.append(df)

            if 'hypoglycemia' in item2.lower() and 'numerator' in item2.lower():
                df['measure'] = 'Hypoglycemia in insulin use'
                df.rename(columns = {'RecordId':'numerator'}, inplace = True) 
                df['numerator_event_type_id'] = 22
                hypo_num_list2.append(df)

            if 'hypoglycemia' in item2.lower() and 'denominator' in item2.lower():
                df['measure'] = 'Hypoglycemia in insulin use'
                df.rename(columns = {'RecordId':'denominator'}, inplace = True) 
                df['denominator_event_type_id'] = 23
                hypo_denom_list2.append(df)

            if 'warfarin' in item2.lower() and 'numerator' in item2.lower():
                df['measure'] = 'Warfarin elevated INR'
                df.rename(columns = {'RecordId':'numerator'}, inplace = True) 
                df['numerator_event_type_id'] = 22
                #print(df.head())
                warfarin_num_list2.append(df)

            if 'warfarin' in item2.lower() and 'denominator' in item2.lower():
                df['measure'] = 'Warfarin elevated INR'
                df.rename(columns = {'RecordId':'denominator'}, inplace = True) 
                df['denominator_event_type_id'] = 23
                #print(df.head())
                warfarin_denom_list2.append(df)

            if 'transfusion' in item2.lower() and 'numerator' in item2.lower():
                df['measure'] = 'Transfusion'
                df.rename(columns = {'RecordId':'numerator'}, inplace = True) 
                df['numerator_event_type_id'] = 22
                trans_num_list2.append(df)

            if 'transfusion' in item2.lower() and 'denominator' in item2.lower():
                df['measure'] = 'Transfusion'
                df.rename(columns = {'RecordId':'denominator'}, inplace = True) 
                df['denominator_event_type_id'] = 23
                trans_denom_list2.append(df)
        except:
            #sometimes Vizient just gives us an empty excel file for transfusion numerators...
            #adding a line to handle this case.
            if 'transfusion' in item2.lower() and 'numerator' in item2.lower():
                df = pd.DataFrame(columns = ['MedicareId','numerator','measure','numerator_event_type_id'])
                trans_num_list2.append(df)
            else:
                print('THERE IS SOMETHING WRONG/DIFFERENT WITH THE FILE.')
                break

#union the dataframes together         
lactate_num_df = pd.concat(lactate_num_list2)
lactate_denom_df = pd.concat(lactate_denom_list2)
hypo_num_df = pd.concat(hypo_num_list2)
hypo_denom_df = pd.concat(hypo_denom_list2)
warfarin_num_df = pd.concat(warfarin_num_list2)
warfarin_denom_df = pd.concat(warfarin_denom_list2)
trans_num_df = pd.concat(trans_num_list2)
trans_denom_df = pd.concat(trans_denom_list2) 


#left join numerator to denominator
lactate_df = pd.merge(lactate_denom_df,lactate_num_df, on='MedicareId',how='left')
hypo_df = pd.merge(hypo_denom_df,hypo_num_df, on='MedicareId',how='left')
trans_df = pd.merge(trans_denom_df,trans_num_df, on='MedicareId',how='left')
warfarin_df = pd.merge(warfarin_denom_df,warfarin_num_df, on='MedicareId',how='left')

final_frames = [lactate_df,hypo_df,trans_df,warfarin_df]
final_df1 = []

for i, item in enumerate(final_frames):
    #Due to left join, if a hospital has zero numerator, they will have a NaN on the right side of the dataframe due to 
    #missing row in the numerator file.  We now need to file these with 0 because this is how Vizient counts.  Denominator
    # not NaN, but NaN numerator = 0.
    item[['numerator']] = item[['numerator']].fillna(value=0)
    item[['numerator_event_type_id']] = item[['numerator_event_type_id']].fillna(value=22)
    item['measure_y'] = item['measure_y'].fillna(item['measure_x'])
    
    item['measure_value'] = item['numerator']/item['denominator']
        
    final_df1.append(item)

    
final_df2 = pd.concat(final_df1)

final_df2 = pd.DataFrame(final_df2, columns = ['MedicareId', 'measure_x', 'measure_value','numerator','numerator_event_type_id','denominator','denominator_event_type_id'])  

final_df2['calc_id'] = calculator_id


final_df2 = final_df2.merge(lab_measure_df, left_on='measure_x', right_on='measure_name')


final_df2['MedicareId'] = final_df2['MedicareId'].astype(str).str.replace(' ','')
hospital_df['hospital_medicare_id'] = hospital_df['hospital_medicare_id'].astype(str).str.replace(' ','')


final_df3 = final_df2.merge(hospital_df, left_on='MedicareId', right_on='hospital_medicare_id',how='left')

final_df3 = final_df3.drop_duplicates()


final_df3['period_id'] = period_query_results
final_df3['measure_value_id'] = 1

final_df3 = final_df3[['calc_id','hospital_id','measure_id','period_id','measure_value_id','measure_value','numerator','numerator_event_type_id','denominator','denominator_event_type_id']]

#final_df3.to_csv('lab_test_02.21.2021.csv',index=False)


P:\Datastore02\Analytics\230 Inpatient Quality Composite\data\Lab Data\fy23\010_june_fytd\Lactate_140062_140242\Lactate_Numerator_PHI.csv
P:\Datastore02\Analytics\230 Inpatient Quality Composite\data\Lab Data\fy23\010_june_fytd\Lactate_140281_149916\Lactate_Numerator_PHI.csv
P:\Datastore02\Analytics\230 Inpatient Quality Composite\data\Lab Data\fy23\010_june_fytd\Lactate_140062_140242\Lactate_Denominator_PHI.csv
P:\Datastore02\Analytics\230 Inpatient Quality Composite\data\Lab Data\fy23\010_june_fytd\Lactate_140281_149916\Lactate_Denominator_PHI.csv
P:\Datastore02\Analytics\230 Inpatient Quality Composite\data\Lab Data\fy23\010_june_fytd\Hypoglycemia_140062_140242\Hypoglycemia_Numerator_PHI.csv
P:\Datastore02\Analytics\230 Inpatient Quality Composite\data\Lab Data\fy23\010_june_fytd\Hypoglycemia_140281_149916\Hypoglycemia_Numerator_PHI.csv
P:\Datastore02\Analytics\230 Inpatient Quality Composite\data\Lab Data\fy23\010_june_fytd\Hypoglycemia_140062_140242\Hypoglycemia_Denominator_PHI.cs

In [22]:
final_df3 = final_df3[~final_df3['hospital_id'].isna()]

In [23]:
final_df3

,calc_id,hospital_id,measure_id,period_id,measure_value_id,measure_value,numerator,numerator_event_type_id,denominator,denominator_event_type_id
0,15,694.0,30,7514,1,0.050239,21.0,22.0,418,23
1,15,472.0,30,7514,1,0.039648,9.0,22.0,227,23
2,15,219.0,30,7514,1,0.053846,21.0,22.0,390,23
3,15,221.0,30,7514,1,0.063444,21.0,22.0,331,23
4,15,112.0,30,7514,1,0.077739,44.0,22.0,566,23
5,15,23.0,30,7514,1,0.069860,35.0,22.0,501,23
6,15,222.0,30,7514,1,0.078125,20.0,22.0,256,23
7,15,471.0,30,7514,1,0.043243,8.0,22.0,185,23
8,15,694.0,158,7514,1,0.032678,113.0,22.0,3458,23
9,15,472.0,158,7514,1,0.014251,29.0,22.0,2035,23


In [24]:
#final_df3 = final_df3[final_df3['hospital_id'] != 694]

In [24]:
def insert_measure_values_from_reports_df(df):
    # connect to the NM_Analytics database
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=edw00pd05wva.corp.nm.org\EDWIDS1;'
                          'Database=NM_Analytics_Prototype;'
                          'Trusted_Connection=yes;')

    cursor = conn.cursor()
    # iterate over df rows and insert into NM_Analytics_Prototype.vizient_qa.datadump
    for index, row in df.iterrows():
        if row['measure_value'] != 'Missing':
            cursor.execute(
                "INSERT INTO NM_Analytics_Prototype.vizient_qa.measure_values([calc_id],[hospital_id],[measure_id],[period_id],[measure_value_id],[measure_value],[numerator],[numerator_event_type_id],[denominator],[denominator_event_type_id]) values (?,?,?,?,?,?,?,?,?,?)",
                row['calc_id'], row['hospital_id'], row['measure_id'], row['period_id'], row['measure_value_id'],
                row['measure_value'], row['numerator'], row['numerator_event_type_id'], row['denominator'],
                row['denominator_event_type_id'])
            conn.commit()

    cursor.close()
    conn.close()
    print('done inserting rows.')

In [26]:
insert_measure_values_from_reports_df(final_df3)

done inserting rows.


In [25]:
final_df3.shape

(35, 10)

In [27]:
final_df3['measure_id'].unique()

array([ 30, 158,  31, 172], dtype=int64)